In [ ]:
import numpy as np
import pandas as pd
import fastf1
from scipy.signal import savgol_filter
from src.plotset import setup_plot, save_fig, plot_track_dominance

from fastf1 import plotting
import matplotlib.pyplot as plt

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,16,'Q')
session.load()

In [ ]:
corners = session.get_circuit_info().corners
rotation = session.get_circuit_info().rotation

In [ ]:
ref_lap = session.laps.pick_drivers('NOR').pick_fastest().get_telemetry().copy()
comp_lap = session.laps.pick_drivers('LEC').pick_fastest().get_telemetry().copy()

In [ ]:
mult = ref_lap.Distance.iloc[-1]/comp_lap.Distance.iloc[-1]

In [ ]:
ref_dist = ref_lap.Distance.to_numpy()
ref_time = ref_lap.Time.dt.total_seconds().to_numpy()
comp_dist = (comp_lap.Distance * mult).to_numpy()
comp_time = comp_lap.Time.dt.total_seconds().to_numpy()

In [ ]:
comp_time = np.interp(ref_dist, comp_dist, comp_time)

In [ ]:
delta = comp_time - ref_time

In [ ]:
dominance = pd.Series(delta).diff().rolling(window=5,center=True).mean()

In [ ]:
setup_plot(axeslabel=24, figtitle=28, legendfont=22)

fig, ax = plt.subplots(2,1,figsize=(15,12),height_ratios=[0.7,0.3],sharex=True)
ax[0].plot(ref_lap['Distance'],ref_lap['Speed'],color="#FF8000",linewidth=3, label='NORRIS')
ax[0].plot(comp_lap['Distance'],comp_lap['Speed'],color="#FF0000",linewidth=3, label='LECLERC')

ax[0].vlines(x=corners['Distance'], ymin=0, ymax=380,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax[0].text(corner['Distance'], 390, txt,
            va='center_baseline', ha='center', size=16, color="#00FF0D")

# ax[0].set_title('Speed Trace', pad=30)
ax[0].set_xlim(ref_dist[0],ref_dist[-1])
ax[0].set_ylim(0, 400)
ax[0].set_ylabel('Speed (km/h)')
ax[0].legend()
ax[0].grid(visible=False)
ax02 = ax[0].twinx()
ax02.set_yticks([])
ax02.set_ylabel('BK',color="#292929",labelpad=30)

ax[1].plot(ref_lap['Distance'],delta,ls='--',lw=3,color='w')
ax[1].axhline(y=0,lw=1,ls='--',color="#FFFFFF")

ax[1].vlines(x=corners['Distance'], ymin=-0.5, ymax=0.5,
          linestyles='dotted', colors='grey')

ax[1].fill_between(x=(ref_dist[0],ref_dist[-1]),y1=0,y2=0.5,color='#FF8000',alpha=0.2)
ax[1].fill_between(x=(ref_dist[0],ref_dist[-1]),y1=-0.5,y2=0,color='#FF0000',alpha=0.2)

# ax[1].set_title('Lap Time Delta')
ax[1].set_xlim(ref_dist[0],ref_dist[-1])
ax[1].set_ylim(-0.4,0.4)
ax[1].set_ylabel('Delta (sec)')
ax[1].set_xlabel('Lap Distance (m)')
ax[1].grid(visible=False)

fig.align_labels(axs=ax)

In [ ]:
fig.savefig(fname='./media/Post8/speed_trace.png',transparent=True,dpi=300)

In [ ]:
setup_plot(axeslabel=20, figtitle=28, legendfont=22)

fig, ax = plt.subplots(4,1,figsize=(15,12),sharex=True)

# RPM
ax[0].plot(ref_lap['Distance'],ref_lap['RPM'],color="#FF8000",linewidth=3, label='NORRIS')
ax[0].plot(comp_lap['Distance'],comp_lap['RPM'],color="#FF0000",linewidth=3, label='LECLERC')

ax[0].vlines(x=corners['Distance'], ymin=5000, ymax=13000,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax[0].text(corner['Distance'], 13500, txt,
            va='center_baseline', ha='center', size=16, color="#00FF0D")

ax[0].set_xlim(ref_dist[0],ref_dist[-1])
ax[0].set_ylim(6000, 14000)
ax[0].set_yticks([6000,8000,10000,12000,14000])
ax[0].set_yticklabels(['6k','8k','10k','12k','14k'])
ax[0].set_ylabel('RPM')
ax[0].legend()
ax[0].grid(visible=False)

# Gear
ax[1].plot(ref_lap['Distance'],ref_lap['nGear'],color="#FF8000",linewidth=3, label='NORRIS')
ax[1].plot(comp_lap['Distance'],comp_lap['nGear'],color="#FF0000",linewidth=3, label='LECLERC')
ax[1].set_ylim(1.5, 8.5)
ax[1].set_ylabel('Gear')
ax[1].vlines(x=corners['Distance'], ymin=1.5, ymax=8.5,
          linestyles='dotted', colors='grey')
ax[1].grid(visible=False)

# Throttle
ax[2].plot(ref_lap['Distance'],ref_lap['Throttle'],color="#FF8000",linewidth=3, label='NORRIS')
ax[2].plot(comp_lap['Distance'],comp_lap['Throttle'],color="#FF0000",linewidth=3, label='LECLERC')
ax[2].set_ylim(-2, 102)
ax[2].set_ylabel('Throttle (%)')
ax[2].vlines(x=corners['Distance'], ymin=-2, ymax=102,
          linestyles='dotted', colors='grey')
ax[2].grid(visible=False)

# Brake
ax[3].plot(ref_lap['Distance'],ref_lap['Brake'],color="#FF8000",linewidth=3, label='NORRIS')
ax[3].plot(comp_lap['Distance'],comp_lap['Brake'],color="#FF0000",linewidth=3, label='LECLERC')
ax[3].set_ylim([-0.2,1.2])
ax[3].set_ylabel('Brakes')
ax[3].set_yticks([0,1])
ax[3].set_yticklabels(['OFF','ON'])
ax[3].vlines(x=corners['Distance'], ymin=-0.1, ymax=1.2,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax[3].text(corner['Distance'], -0.15, txt,
            va='center_baseline', ha='center', size=16, color="#00FF0D")
ax[3].set_xlabel('Lap Distance (m)')
ax[3].grid(visible=False)
fig.align_labels(axs=ax)

In [ ]:
fig.savefig(fname='./media/Post8/other_tel.png',transparent=True,dpi=300)

In [ ]:
def long_acc(lap, window=6, polyorder=2):

    dv = (lap.Speed * (5/18)).diff()
    dt = lap.Time.dt.total_seconds().diff()
    a = dv / dt
    a_g = a / 9.81

    # Smooth with Savitzky–Golay filter
    return savgol_filter(a_g.fillna(0), window_length=window, polyorder=polyorder)

In [ ]:
comp_lap['Distance'] = comp_dist

ref_lap['long_acc'] = long_acc(ref_lap,window=11,polyorder=2)
comp_lap['long_acc'] = long_acc(comp_lap,window=11,polyorder=2)

In [ ]:
def compute_lateral_acc(df, window_length=21, polyorder=2, outlier_threshold=100, xy_window=21, xy_poly=2):

    df = df.copy()

    # --- Pre-filter X, Y, Distance ---
    n = len(df)
    w = min(xy_window, n if n % 2 == 1 else n-1)  # ensure odd, <= n
    if w < 3:
        w = 3

    x = savgol_filter(df['X'].to_numpy(), w, xy_poly)
    y = savgol_filter(df['Y'].to_numpy(), w, xy_poly)
    s = savgol_filter(df['Distance'].to_numpy(), w, xy_poly)

    # Step 1. Heading angle
    delta_x = np.diff(x, prepend=x[0])
    delta_y = np.diff(y, prepend=y[0])
    heading = np.arctan2(delta_y, delta_x)

    # Step 2. Heading change and distance delta
    d_heading = np.diff(heading, prepend=heading[0])
    ds = np.diff(s, prepend=s[0])

    # Step 3. Radius of curvature
    radius = np.divide(ds, d_heading, 
                       out=np.full_like(ds, np.nan, dtype=float), 
                       where=d_heading != 0)

    # Step 4. Lateral acceleration (v²/R)
    v = df['Speed'].to_numpy() / 3.6  # km/h → m/s
    lat_acc = np.divide(v**2, radius, 
                        out=np.full_like(v, np.nan, dtype=float),
                        where=~np.isnan(radius))

    # Step 5. Remove unrealistic values
    lat_acc[np.abs(lat_acc) > outlier_threshold] = np.nan

    # Step 6. Smooth LatAcc and convert to g
    lat_acc_filt = savgol_filter(np.nan_to_num(lat_acc), 
                                 window_length=window_length, 
                                 polyorder=polyorder)

    return lat_acc_filt / 9.81  # in g

In [ ]:
ref_lap['lat_acc'] = compute_lateral_acc(ref_lap)
comp_lap['lat_acc'] = compute_lateral_acc(comp_lap)

In [ ]:
setup_plot(axeslabel=20, figtitle=28, legendfont=22)

fig, ax = plt.subplots(2,1,figsize=(15,12))

ax[0].plot(ref_lap['Distance'],ref_lap['long_acc'],color="#FF8000",linewidth=3,label='NORRIS')
ax[0].plot(comp_lap['Distance'],comp_lap['long_acc'],color="#FF0000",linewidth=3,label='LECLERC')
ax[0].set_xlim(ref_dist[0],ref_dist[-1])
ax[0].set_ylim(-6, 2.5)
ax[0].set_ylabel('Longitudinal Acceleration (g)')
ax[0].legend()

ax[0].vlines(x=corners['Distance'], ymin=-6, ymax=2,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax[0].text(corner['Distance'], 2.2, txt,
            va='center_baseline', ha='center', size=16, color='#00FF0D')
    
ax[0].grid(visible=False)
    

ax[1].plot(ref_lap['Distance'],ref_lap['lat_acc'],color="#FF8000",linewidth=3)
ax[1].plot(comp_lap['Distance'],comp_lap['lat_acc'],color="#FF0000",linewidth=3)
ax[1].set_xlim(ref_dist[0],ref_dist[-1])
ax[1].set_ylim(-6, 4.5)
ax[1].set_ylabel('Lateral Acceleration (g)')
ax[1].set_xlabel('Lap Distance (m)')

ax[1].vlines(x=corners['Distance'], ymin=-6, ymax=4.5,
          linestyles='dotted', colors='grey')

ax[1].grid(visible=False)

In [ ]:
fig.savefig(fname='./media/Post8/acc_tel.png',transparent=True,dpi=300)